In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd
import cv2
from PIL import Image

In [4]:
input_size = 28
num_classes = 10
num_epochs = 10
batch_size = 120

train_dataset = torchvision.datasets.MNIST(root='data',
                              train=True,
                              transform=torchvision.transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='data',
                             train=False,
                             transform=torchvision.transforms.ToTensor())

In [5]:
#构建 mini-batch
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [10]:
#构建 网络结构
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,
                                             out_channels=8,
                                             kernel_size=5,
                                             stride=1,
                                             padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        self.conv2 = nn.Sequential(nn.Conv2d(8, 16, 5, 1, 1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        self.out = nn.Sequential(nn.Flatten(),
                                 nn.Linear(16 * 5 * 5, 64),
                                 nn.ReLU(),
                                 nn.Linear(64, 10),)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        output = self.out(x)
        return output

In [7]:
#定义准确率作为评估标准
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]            #torch.max()[0]返回最大值 torch.max()[1]返回最大值位置，既预测值
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)                          #这里返回的是正确个数与总个数组成的元组

In [23]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.cuda()
test_right = []
train_loss = []

for epoch in range(num_epochs):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                data, target = data.cuda(), target.cuda()
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            test_right.append((100*val_r[0]/val_r[1]).cpu().numpy())
            train_loss.append(loss.data.cpu().numpy())

            print(f'当前epoch：{epoch} [{batch_idx*batch_size}/{len(train_loader.dataset)}]   损失：{loss.data:.6f}  训练集准确率：{100*train_r[0]/train_r[1]:.2f}%   测试集准确率：{100*val_r[0]/val_r[1]:.2f}%')

当前epoch：0 [0/60000]   损失：2.300909  训练集准确率：12.50%   测试集准确率：10.32%
当前epoch：0 [12000/60000]   损失：0.306057  训练集准确率：72.65%   测试集准确率：88.91%
当前epoch：0 [24000/60000]   损失：0.198007  训练集准确率：82.04%   测试集准确率：93.01%
当前epoch：0 [36000/60000]   损失：0.238400  训练集准确率：85.95%   测试集准确率：94.82%
当前epoch：0 [48000/60000]   损失：0.205797  训练集准确率：88.13%   测试集准确率：95.82%
当前epoch：1 [0/60000]   损失：0.075445  训练集准确率：98.33%   测试集准确率：96.38%
当前epoch：1 [12000/60000]   损失：0.107527  训练集准确率：96.53%   测试集准确率：96.89%
当前epoch：1 [24000/60000]   损失：0.134942  训练集准确率：96.73%   测试集准确率：97.34%
当前epoch：1 [36000/60000]   损失：0.090773  训练集准确率：96.85%   测试集准确率：97.93%
当前epoch：1 [48000/60000]   损失：0.070903  训练集准确率：96.99%   测试集准确率：97.40%
当前epoch：2 [0/60000]   损失：0.055193  训练集准确率：99.17%   测试集准确率：98.03%
当前epoch：2 [12000/60000]   损失：0.141378  训练集准确率：97.71%   测试集准确率：98.13%
当前epoch：2 [24000/60000]   损失：0.048735  训练集准确率：97.74%   测试集准确率：98.35%
当前epoch：2 [36000/60000]   损失：0.164965  训练集准确率：97.85%   测试集准确率：98.22%
当前epoch：2 [48000/60000]   损失：0.040464  训练集准确率：

In [26]:
loss_ = pd.DataFrame()
ac_ = pd.DataFrame()
loss_['relu'] = train_loss
ac_['relu'] = test_right